In [50]:
import pandas as pd
import numpy as np

from scipy import stats

from statsmodels.sandbox.stats.multicomp import multipletests 

In [4]:
rp = pd.read_csv('/home/yukos/work/hse/applied_stats/week4/religion_politics.csv')

In [6]:
rp.politics.unique()

array(['Conservative', "Don't know", 'Slghtly conservative', 'Moderate',
       'Extrmly conservative', 'Slightly liberal', 'Liberal',
       'Extremely liberal'], dtype=object)

In [33]:
ct = pd.crosstab(rp['religion'], rp['politics'], margins=True)
ct

politics,Conservative,Don't know,Extremely liberal,Extrmly conservative,Liberal,Moderate,Slghtly conservative,Slightly liberal,All
religion,,,,,,,,,
Buddhism,3,0,3,0,5,4,0,4,19
Catholic,73,21,8,20,38,203,70,56,489
Christian,8,0,2,1,2,16,4,2,35
Hinduism,0,0,0,0,2,5,0,1,8
Jewish,7,1,4,1,11,11,2,2,39
Moslem/islam,3,1,0,0,2,8,0,2,16
None,38,22,56,9,109,189,40,74,537
Other,1,1,0,0,9,18,3,3,35
Protestant,219,30,49,66,97,395,164,111,1131


1. Calculate the matrix of expected counts under independence. What is the expected number of conservative protestants? Provide the answer rounded to 2 decimal points.

In [13]:
(352 * 1131)/2309

172.41749675184062

2.Calculate standardized Pearson residuals for the contingency table. What is the value for liberal catholics? Provide the answer rounded to 3 decimal points.

In [27]:
n = 2309
ni = 489
nj = 275

In [28]:
eij = (ni * nj) / n
eij

58.23949761801646

In [29]:
nij = 38

In [30]:
np.sqrt(eij * (1 - ni/n) * (1 - nj/n))

6.359105723027162

In [32]:
(nij - eij) / np.sqrt(eij * (1 - ni/n) * (1 - nj/n))

-3.182758472583112

Each residual has standard normal distribution if the observed count is in line with the expected one. Let's use them to test the hypotheses that each observed count comes from a distribution with the mean equal to the corresponding expected count, against two-sided alternative.

Calculate p-values for all cells of the contingency tables. How many of them are less than or equal to 0.05?

In [47]:
num_rows = ct.shape[0] - 1
num_columns = ct.shape[1] - 1

In [49]:
m = []
for i in range(num_rows):
    ar = []
    for j in range(num_columns):
        nij = ct.iloc[i, j]
        eij = (ct.iloc[i, -1] * ct.iloc[-1, j]) / n
        pearson = (nij - eij) / np.sqrt(eij * (1 - ni/n) * (1 - nj/n))
        ar.append(pearson)
    
    m.append(ar)

ps = np.array(m)
ps

array([[ 0.07298785, -0.94903973,  2.39084153, -1.07216954,  2.18360826,
        -1.35582745, -1.83134774,  1.57549614],
       [-0.21496358,  1.46716103, -4.21129892, -0.14368505, -3.18275847,
         2.07626002,  1.56043374,  0.32597402],
       [ 1.3842412 , -1.28807638,  0.13300443, -0.46549041, -1.27461205,
         1.04734836, -0.16788028, -1.13860099],
       [-1.32530893, -0.6158183 , -0.7802355 , -0.69571547,  1.2874956 ,
         1.44035576, -1.18833533,  0.1487435 ],
       [ 0.51903438, -0.3004713 ,  1.62133467, -0.59852199,  3.53875636,
        -1.0584776 , -1.52595825, -1.33407763],
       [ 0.43096226,  0.78280605, -1.10341963, -0.98389025,  0.08207879,
         1.04741348, -1.68055994,  0.21035507],
       [-5.81801173,  1.23452083,  6.22424017, -3.42596651,  6.75936195,
        -0.72170813, -3.81897526,  2.290025  ],
       [-2.25254204, -0.16996797, -1.63197964, -1.4551933 ,  2.8399385 ,
         1.71641112, -0.74730563, -0.52819243],
       [ 4.25740455, -1.42139551

In [62]:
np.sum((stats.norm.cdf(ps) < 0.025) | (stats.norm.cdf(ps) > 0.975))


18